# Web Scrapping lab

In this lab you will scrappe this [website](https://books.toscrape.com/) of books.

You have to create a Pandas DataFrame with all the books listed in the page. Each row of the DataFrame should contain information of each book. In particular, the DataFrmae must contain:

* category
* title
* price
* stock availability
* star rating (number of stars)
* description
* UPC

Happy scrapping!



# Server verification

Load the needed libraries, and make sure thar you can obtain the correct status code.

In [1]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://books.toscrape.com/"
response = requests.get(url)


# Book categories

Create the code to collect the **relative urls** from the left panel to obtain a list with all the book categories.

In [2]:

soup = BeautifulSoup(response.content, "html.parser")
categories = soup.find("ul", class_="nav").find_all("a")

cat_name = []
cat_url = []

for category in categories:
    cat_name.append(category.text.strip())
    cat_url.append(category["href"])



# Books in a given category

Use. web scrapping and list comprehension to obtain the **absolute** url of each book to be scraped

In [ ]:
url_list = []
for x in range(1, 50):
    response = requests.get(f"https://books.toscrape.com/catalogue/page-{x}.html")
    soup = BeautifulSoup(response.content, "html.parser")
    url_book = soup.find("ol", class_="row").find_all("a")

    for y in range(0, len(url_book), 2):
        url_list.append("https://books.toscrape.com/catalogue/" + url_book[y]["href"])


# Book details

Create a Python function that given a book_url as an input returns a dictionary with the following structure:

```Python
{"Title": title, "Price": price, "Availability": availability, "Rating": rating, "Description": description, "UPC": upc}
```

where `description` should contain the book's summary given in the Product description, and the values are the book's associated information.

In [ ]:


def book_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    title = soup.find('h1').string
    price = soup.find('p', class_='price_color')
    availability = soup.find('p', class_='instock availability')
    rating = soup.find('p', class_='star-rating')
    description = soup.find('meta', attrs={"name": "description"})
    upc = soup.find('th', text='UPC')

    dic = {
        "Title": title,
        "Price": price,
        "Availability": availability,
        "Rating": rating,
        "Description": description,
        "UPC": upc
    }
    
    return dic



# Collect and store all the information from the books in a Pandas DataFrame

Start with the following dictionary:

```python
books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }
```

Then, iterate over all the categories and all the books in a given category to collect any book information using the previous function. Fill the previous dictionary with the information about each book.

Show the first five rows of the previous final Pandas DataFrame.

Tip: You can use the function `tqdm` from the library `tqdm` to show a progress bar if in iterable of a for loop as shown below :wink: :

```python
from tqdm import tqdm

for elem in tqdm(iterable):
    # some code
```





In [ ]:
from tqdm import tqdm

books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }

for category_name, category in tqdm(zip(cat_name, cat_url)):
    category_url = url + category
    response = requests.get(category_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    books = soup.find_all('article', class_='product_pod')
    book_urls = [url + 'catalogue/' + book.h3.a['href'] for book in books]

    for book_url in book_urls:
        book_info = book_details(book_url)
        books_dict["Title"].append(book_info["Title"])
        books_dict["Price"].append(book_info["Price"])
        books_dict["Availability"].append(book_info["Availability"])
        books_dict["Rating"].append(book_info["Rating"])
        books_dict["Description"].append(book_info["Description"])
        books_dict["UPC"].append(book_info["UPC"])
        books_dict["Category"].append(category_name)

# Creating DataFrame from the dictionary
df = pd.DataFrame(books_dict)

# Display the DataFrame
print(df)